In [86]:
#HDD:
  #  HDDs=∑HDDi=∑max（0，Tref -Ti）
   # Ti=T_Average
    #Tref=T_standard
    #Tref standard 18 c which 65 F
    
#St:
    #St(Call)=exp[-r(T-t）]E｛min［Cmax（HDDs（0，t） +HDDs（t，T）-K，0），U］｝

In [87]:
import numpy as np
import math
import csv

import numpy as np
from random import gauss
from math import exp, sqrt

In [88]:
Tref=18

In [89]:
#Beijing
#Tref=18

with open('Beijing.csv')as Beijing:
    Beijing_csv=csv.reader(Beijing)
    HDD_s=0
    Date=[]
    Ti=[]
    for row in Beijing_csv:
        Date.append(row[0])
        Ti.append(row[2])

In [90]:
for n in range(1,len(Ti)):
    Date_str=str(Date[n])
    year=Date_str.split("/")[0]
    month=Date_str.split("/")[1]

In [91]:
#"2019/1/1"    25343
#"2019/7/1"    25524
#"2020/1/1"    25708
#"2020/7/1"    25890
#"2021/1/1"    26074
#"2021/7/1"    26252


In [92]:
a=[25343,25524,25708,25890,26074]
b=[25524,25708,25890,26074,26252]

In [95]:
#字段名称
fields = ['DATE','TAVG','HDDi','HDDs']

#csv文件的名字
filename = 'BeijingHDD.csv'

#写入csv文件
with open(filename,'w') as csvfile:
    csvwriter = csv.writer(csvfile)

    #写入字段
    csvwriter.writerow(fields)
    
#19年开始
    
    
    rows=[]
    HDD_0t=[]
    for n in range (5):
        beg=a[n]
        end=b[n]
        HDDs=0
        for i in range(beg,end):
            Ti_num=float(Ti[i])
            HDDi=max(0,Tref-Ti_num)
            HDDs=HDDi+HDDs

            row=[]
            row.append(Date[i])
            row.append(Ti[i])
            row.append(HDDi)
            row.append(HDDs)

            rows.append(row)

        csvwriter.writerows(rows)
        



In [94]:
Data_s=[2019.0101, 2019.0701, 2020.0101, 2020.0701, 2021.0101 ]

In [79]:
HDD_0t=[]

In [80]:
with open('BeijingHDD.csv')as BeijingHDD:
    BeijingHDD_csv=csv.reader(BeijingHDD)
    
    hdds=[]
    for row in BeijingHDD:
        
        hdds.append(row.split(",")[3])
        
    HDD_0t.append(hdds[len(hdds)-1])
    

In [69]:
print(HDD_0t)

['1404.875\n']


In [54]:
##HDDs
#2019.1.1==1404.875   
#2019.7.1==1063.5
#2020.1.1==1399   
#2020.7.1==1115.125
#2021.1.1==1390.25   

In [55]:
#Data_s=[2019.0101, 2019.0701, 2020.0101, 2020.0701, 2021.0101 ]
#HDDs_0t=[1404.875,1063.5,1399,1115.125,1390.25 ]

In [56]:
## 准备工作
import numpy as np
import math

## 初始化关键参数的值
#S0：标的物在时间t的价格水平；为[HDDs(0,t)]
#   σ：标的物收益率的波动率，可理解为它的收益率的标准差0.51。
#   K：期权的行权价格；我们假设为100（参考现值有利法，设定为低于历年同期水平的价格）
#   T：期权到期日；假设为半年（0.5）。
#   r：无风险利率；假设为2.72%。


def HDD_tT(S0):
    sigma=0.51
    K=100
    T=0.5
    r=0.0272
    I=1000000 ##I个随机数。重复I次

    ## 估值
    z = np.random.standard_normal(I) ##返回1000000个服从标准正太分布的随机小数
    ST = S0 * np.exp((r-0.5*sigma**2)*T + sigma*math.sqrt(T)*z)
    hT = np.maximum(ST-K, 0) ##到期的回报
    return math.exp(-r*T)*np.mean(hT) ##蒙特卡洛模拟估计值
    #HDD_tT=co
    

In [57]:
HDD_tT(1404.875)

1306.863460138502

In [58]:
1-(HDD_tT(1404.875)/1404.875)

0.07044253785879573

In [59]:
#字段名称
fields = ['基线温度','起始日期','实际指数','预测指数','相对误差']

#csv文件的名字
filename = 'Beijing-总结.csv'

#写入csv文件
with open(filename,'w') as csvfile:
    csvwriter = csv.writer(csvfile)

    #写入字段
    csvwriter.writerow(fields)
    
    rows=[]
    for i in range(5):
        row=[]
        row.append(Tref)#Tref为基线温度
        row.append(Data_s[i])
        row.append(HDDs_0t[i])
        #调用HDD_tT
        HDD_T=HDD_tT(HDDs_0t[i])
        row.append(HDD_T)
        #误差
        re_error=1-(HDD_T/HDDs_0t[i])
        row.append(re_error)
        
        rows.append(row)
        
    csvwriter.writerows(rows)
    


In [ ]:
#St:
    #St(Call)=exp[-r(T-t）]E｛min［Cmax（HDDs（0，t） +HDDs（t，T）-K，0），U］｝

In [137]:
def calculate_S_T(S, v, r, T):
    """模拟epsilon，计算S_T"""
    return S * exp((r - 0.5 * v ** 2) * T + v * sqrt(T) * gauss(0.0, 1.0))

def option_payoff(flag, S_T, K):
    """期权收益"""
    if flag == 'call':
        return max(S_T - K, 0.0)
    else:
        return max(K - S_T, 0.0)

def option_price(flag, S, r, T, v, K, simulations):
    """模拟"""
    payoffs = []
    discout = exp(-r * T)
    for i in range(simulations):
        S_T = calculate_S_T(S, v, r, T)
        payoffs.append(
            option_payoff(flag, S_T, K)
        )
    price = discout * sum(payoffs) / float(simulations)
    return price

In [139]:
S = 276.10
r = 2.72/100
T=0.5
v = 0.407530933
K = 250
simulations = 1000000
print(option_price('call', S, r, T, v, K, simulations))
print(option_price('put', S, r, T, v, K, simulations))

79.97670085095626
40.503321133147345
